# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Описание проекта**
- Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
- В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
- Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

## Обзор данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier


In [ ]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [ ]:
data.head(15)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [ ]:
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [ ]:
data.duplicated().sum()

0

**Вывод**
- Проблем в данных не было обнаружено, можно преступать к дальнейшим шагам

Необходимо разбить данные на выборки:
- Обучающая выборка
- Валидационная выборка
- Тестовая выборка

In [ ]:
data_train, data_valid = train_test_split(data, test_size=0.4, random_state=12345)

In [ ]:
data_valid, data_test = train_test_split(data_valid, test_size=0.6, random_state=12345)

**Выделяем целевой признак**

In [ ]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

In [ ]:
data_train.shape

(1928, 5)

In [ ]:
data_valid.shape

(514, 5)

In [ ]:
data_test.shape

(772, 5)

## Исследуйте модели

**Дерево решений**

In [ ]:
best_model = None
best_result = 0
best_depth = 0

for depth in range(1, 20):
    model = DecisionTreeClassifier(max_depth=depth, random_state=1234)
    model.fit(features_train , target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
print("Глубина дерева:", best_depth, "Лучший результат качества:", best_result)

Глубина дерева: 3 Лучший результат качества: 0.7996108949416343


**Случайный лес**

In [ ]:
best_model = None
best_est = 0
best_depth = 0
best_result = 0
for est in range(10, 20, 200):
    for depth in range(10, 20, 200):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if best_result < result:
            best_model = model
            best_est = est
            best_depth = depth
            best_result = result
print('Количество деревьев:', best_est, 'Глубина дерева:', best_depth, 'Лучший результат качества:', best_result)

Количество деревьев: 10 Глубина дерева: 10 Лучший результат качества: 0.8093385214007782


**Логическая регрессия**

In [ ]:
model = LogisticRegression()
model.fit(features_train , target_train)
result = model.score(features_valid, target_valid)
print("Лучший результат качества:", result)

Лучший результат качества: 0.7062256809338522


**Вывод**

После исследования модели мы получили следующие результаты:
- Качество модели логической регрессии - 0.7062256809338522
- Качество модели дерево решений - 0.7996108949416343
- Качество модели случайный лес - 0.8171206225680934
- Исходя из этого лучше выбрать модель - случаный лес

## Проверьте модель на тестовой выборке

In [ ]:
model = RandomForestClassifier(max_depth=8, n_estimators=9, random_state=1234)
model.fit(features_train , target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7823834196891192


## Проверьте модели на адекватность

**Для проверки выбираем модель DummyClassifier со стратегией most_frequent**

In [ ]:
dummy = DummyClassifier(strategy= "most_frequent", random_state=0)
dummy.fit(features_valid, target_valid)
dummy.score(features_valid, target_valid)

0.7023346303501945

## Итоговый вывод

- Проверены три модели: деревья решений, случайный лес, логистическая регрессия
- Лучшей моделью по качеству стала модель случайный лес с гиперпараметрами max_depth=8, n_estimators=9
- Также была проведена проверка модели случайный лес на адекватность, что подтвердилось меньшим значением модели DummyClassifier